In [ ]:
%pip install bs4

In [ ]:
from bs4 import BeautifulSoup
import requests

URL = "http://www.wikipedia.org"
response = requests.get(URL)
print(response.content)
print(response.text)
print(response.status_code)
response.raise_for_status()


In [ ]:
soup = BeautifulSoup(response.text, "html.parser")
print(soup.prettify())

print(soup.title)
print(soup.title.text)
print(soup.title.get_text()[0::1])

print(soup.h1)
print(soup.h1.parent.name)

print(soup.body.h1.previous_sibling)
print(soup.body.h1.attrs['class'][0])

print(soup.find_all("a"))
print(soup.find(class_="link-box"))
print(list(soup.body.div.stripped_strings))

In [ ]:
import re

def returnMatches(target):
    return re.findall("[a-zA-Z]+", target)


print(returnMatches(soup.body.div.text))


In [ ]:
print(soup.select("h1 > span")[0])

In [ ]:
import pickle
import os

filename = "wikipedia-scraped-data.pickle"
if os.path.exists(filename):
    with open(filename, 'rb') as f:
        response = pickle.load(f)

else: 
    result = requests.get(URL)
    with open(filename, "wb") as f:
        pickle.dump(result, f)


print(result.text[0:100])

In [56]:
import csv

row = {"First Name": "Ibrahim", "Last Name": "Rehman", "Age": 20}

with open("wikipedia-scraped-data.csv", "w") as f:
    writer = csv.DictWriter(f, fieldnames=row.keys())
    writer.writeheader()
    writer.writerow(row)